# Luku 7: Keskustelusovellusten rakentaminen
## Github Models API pikaopas

Tämä muistikirja on muokattu [Azure OpenAI Samples Repository](https://github.com/Azure/azure-openai-samples?WT.mc_id=academic-105485-koreyst) -kokoelmasta, joka sisältää muistikirjoja [Azure OpenAI](notebook-azure-openai.ipynb) -palveluiden käyttöön.


# Yleiskatsaus  
"Suuret kielimallit ovat funktioita, jotka muuntavat tekstiä tekstiksi. Kun niille annetaan syötteenä tekstijono, suuri kielimalli yrittää ennustaa, mikä teksti tulee seuraavaksi."(1). Tämä "pika-aloitus" -muistikirja esittelee käyttäjille korkean tason LLM-käsitteitä, ydinkirjastot, joita tarvitaan AML:n käyttöönottoon, kevyen johdatuksen kehotteiden suunnitteluun sekä useita lyhyitä esimerkkejä erilaisista käyttötapauksista.


## Sisällysluettelo  

[Yleiskatsaus](../../../../07-building-chat-applications/python)  
[Kuinka käyttää OpenAI-palvelua](../../../../07-building-chat-applications/python)  
[1. OpenAI-palvelun luominen](../../../../07-building-chat-applications/python)  
[2. Asennus](../../../../07-building-chat-applications/python)    
[3. Tunnistetiedot](../../../../07-building-chat-applications/python)  

[Käyttötapaukset](../../../../07-building-chat-applications/python)    
[1. Tekstin tiivistäminen](../../../../07-building-chat-applications/python)  
[2. Tekstin luokittelu](../../../../07-building-chat-applications/python)  
[3. Uusien tuotenimien generointi](../../../../07-building-chat-applications/python)  
[4. Luokittelijan hienosäätö](../../../../07-building-chat-applications/python)  

[Viitteet](../../../../07-building-chat-applications/python)


### Luo ensimmäinen kehotteesi  
Tämä lyhyt harjoitus antaa perustiedot siitä, miten voit lähettää kehotteita mallille Github Modelsissa yksinkertaista tehtävää, kuten "yhteenvetoa", varten.

**Vaiheet**:  
1. Asenna `azure-ai-inference`-kirjasto Python-ympäristöösi, jos et ole vielä tehnyt niin.  
2. Lataa tavalliset apukirjastot ja määritä Github Models -tunnukset.  
3. Valitse malli tehtävääsi varten  
4. Luo yksinkertainen kehotus mallille  
5. Lähetä pyyntösi mallin API:lle!


### 1. Asenna `azure-ai-inference`


In [ ]:
%pip install azure-ai-inference

### 2. Tuo apukirjastot ja luo tunnistetiedot


In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

### 3. Oikean mallin löytäminen  
GPT-3.5-turbo- tai GPT-4-mallit pystyvät ymmärtämään ja tuottamaan luonnollista kieltä.


In [ ]:
# Select the General Purpose curie model for text
model_name = "gpt-4o"

## 4. Kehotteen suunnittelu  

"Suurten kielimallien taika on siinä, että kun niitä opetetaan minimoimaan ennustusvirhe valtavien tekstimäärien avulla, mallit oppivat samalla käsitteitä, jotka auttavat näissä ennustuksissa. Esimerkiksi ne oppivat käsitteitä kuten"(1):

* miten kirjoitetaan oikein
* miten kielioppi toimii
* miten ilmaista asiat toisin sanoin
* miten vastata kysymyksiin
* miten käydä keskustelua
* miten kirjoittaa monilla kielillä
* miten koodata
* jne.

#### Miten ohjata suurta kielimallia  
"Kaikista suuren kielimallin syötteistä ylivoimaisesti merkittävin on tekstikehote(1).

Suuria kielimalleja voidaan ohjata tuottamaan haluttua tulosta muutamalla tavalla:

Ohjeistus: Kerro mallille, mitä haluat
Täydennys: Johdata malli täydentämään haluamasi alku
Esimerkki: Näytä mallille, mitä haluat, joko:
Muutama esimerkki kehotteessa
Satoja tai tuhansia esimerkkejä hienosäädön harjoitusaineistossa"



#### Kolme perusohjetta kehotteiden luomiseen:

**Näytä ja kerro**. Tee selväksi, mitä haluat, joko ohjeilla, esimerkeillä tai näiden yhdistelmällä. Jos haluat mallin laittavan listan aakkosjärjestykseen tai luokittelemaan kappaleen tunnetilan mukaan, näytä sille, että tätä haluat.

**Käytä laadukasta aineistoa**. Jos yrität rakentaa luokittelijaa tai saada mallin noudattamaan tiettyä kaavaa, varmista, että esimerkkejä on riittävästi. Tarkista esimerkkisi huolellisesti — malli on yleensä tarpeeksi fiksu huomaamaan perus kirjoitusvirheet ja antaa silti vastauksen, mutta se saattaa myös olettaa virheiden olevan tarkoituksellisia, mikä voi vaikuttaa vastaukseen.

**Tarkista asetukset.** Temperature- ja top_p-asetukset säätelevät, kuinka määrätietoisesti malli tuottaa vastauksen. Jos haluat vastauksen, johon on vain yksi oikea vaihtoehto, kannattaa nämä asettaa matalalle. Jos taas haluat monipuolisempia vastauksia, voit nostaa arvoja. Yleisin virhe näiden asetusten kanssa on olettaa, että ne säätelevät "älykkyyttä" tai "luovuutta".


Lähde: https://learn.microsoft.com/azure/ai-services/openai/overview


In [ ]:
# Create your first prompt
text_prompt = "Should oxford commas always be used?"

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

In [ ]:

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":text_prompt},])

response.choices[0].message.content

## Tiivistä teksti  
#### Haaste  
Tiivistä teksti lisäämällä 'tl;dr:' tekstikappaleen loppuun. Huomaa, miten malli osaa suorittaa useita tehtäviä ilman lisäohjeita. Voit kokeilla kuvaavampia kehotteita kuin tl;dr muokataksesi mallin toimintaa ja räätälöidäksesi tiivistelmän haluamallasi tavalla(3).  

Viimeaikaiset tutkimukset ovat osoittaneet merkittäviä edistysaskeleita monissa NLP-tehtävissä ja -mittareissa, kun ensin esikoulutetaan suurella tekstikorpuksella ja sen jälkeen hienosäädetään tiettyyn tehtävään. Vaikka arkkitehtuuri on yleensä tehtäväriippumaton, tämä menetelmä vaatii silti tuhansien tai jopa kymmenientuhansien esimerkkien tehtäväkohtaisia hienosäätöaineistoja. Ihmiset sen sijaan pystyvät yleensä suorittamaan uuden kielitehtävän vain muutaman esimerkin tai yksinkertaisten ohjeiden avulla – tässä nykyiset NLP-järjestelmät ovat vielä pitkälti haasteiden edessä. Tässä osoitamme, että kielimallien skaalaaminen parantaa huomattavasti tehtäväriippumatonta, vähäesimerkkistä suorituskykyä, ja joskus jopa yltää kilpailemaan aiempien huipputason hienosäätömenetelmien kanssa.



Tl;dr


# Harjoituksia eri käyttötarkoituksiin  
1. Tiivistä teksti  
2. Luokittele teksti  
3. Luo uusia tuotenimiä


In [ ]:
prompt = "Recent work has demonstrated substantial gains on many NLP tasks and benchmarks by pre-training on a large corpus of text followed by fine-tuning on a specific task. While typically task-agnostic in architecture, this method still requires task-specific fine-tuning datasets of thousands or tens of thousands of examples. By contrast, humans can generally perform a new language task from only a few examples or from simple instructions - something that current NLP systems still largely struggle to do. Here we show that scaling up language models greatly improves task-agnostic, few-shot performance, sometimes even reaching competitiveness with prior state-of-the-art fine-tuning approaches.\n\nTl;dr"


In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Luokittele teksti  
#### Haaste  
Luokittele kohteet kategorioihin, jotka annetaan vasta ennustevaiheessa. Seuraavassa esimerkissä annamme sekä kategoriat että luokiteltavan tekstin kehotteessa (*playground_reference).

Asiakaskysely: Hei, yksi kannettavani näppäimistä meni rikki hiljattain ja tarvitsisin siihen uuden:

Luokiteltu kategoria:


In [ ]:
prompt = "Classify the following inquiry into one of the following: categories: [Pricing, Hardware Support, Software Support]\n\ninquiry: Hello, one of the keys on my laptop keyboard broke recently and I'll need a replacement:\n\nClassified category:"
print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt},])

response.choices[0].message.content

## Luo uusia tuotenimiä
#### Haaste
Luo tuotenimiä esimerkkisanojen pohjalta. Tässä annamme kehotteessa tietoa tuotteesta, jolle haluamme keksiä nimiä. Mukana on myös vastaava esimerkki, joka näyttää toivotun mallin. Olemme lisäksi asettaneet temperature-arvon korkeaksi, jotta vastauksista tulisi sattumanvaraisempia ja luovempia.

Tuotekuvaus: Kotikäyttöinen pirtelökone
Siemensanat: nopea, terveellinen, kompakti.
Tuotenimet: HomeShaker, Fit Shaker, QuickShake, Shake Maker

Tuotekuvaus: Kengät, jotka sopivat kaikenkokoisiin jalkoihin.
Siemensanat: mukautuva, istuvuus, omni-fit.


In [ ]:
prompt = "Product description: A home milkshake maker\nSeed words: fast, healthy, compact.\nProduct names: HomeShaker, Fit Shaker, QuickShake, Shake Maker\n\nProduct description: A pair of shoes that can fit any foot size.\nSeed words: adaptable, fit, omni-fit."

print(prompt)

In [ ]:
#Setting a few additional, typical parameters during API Call

response = client.complete(
  model=model_name,
  messages = [{"role":"system", "content":"You are a helpful assistant."},
               {"role":"user","content":prompt}])

response.choices[0].message.content

# Viitteet  
- [Openai Cookbook](https://github.com/openai/openai-cookbook?WT.mc_id=academic-105485-koreyst)  
- [OpenAI Studio Esimerkit](https://oai.azure.com/portal?WT.mc_id=academic-105485-koreyst)  
- [Parhaat käytännöt GPT-3:n hienosäätöön tekstin luokittelua varten](https://docs.google.com/document/d/1rqj7dkuvl7Byd5KQPUJRxc19BJt8wo0yHNwK84KfU3Q/edit#?WT.mc_id=academic-105485-koreyst)


# Lisäapua varten  
[OpenAI Commercialization Team](AzureOpenAITeam@microsoft.com)


# Avustajat
* [Chew-Yean Yam](https://www.linkedin.com/in/cyyam/)



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Pyrimme tarkkuuteen, mutta huomioithan, että automaattiset käännökset saattavat sisältää virheitä tai epätarkkuuksia. Alkuperäistä asiakirjaa sen alkuperäisellä kielellä tulee pitää ensisijaisena lähteenä. Kriittisissä tapauksissa suosittelemme ammattimaisen ihmiskääntäjän käyttöä. Emme ole vastuussa tämän käännöksen käytöstä mahdollisesti aiheutuvista väärinkäsityksistä tai tulkinnoista.
